<a href="https://colab.research.google.com/github/mbilalnust/spam_sms_text_classification/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import re
import string
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                    '[%s]' % re.escape(string.punctuation),
                                    '')

In [ ]:
# Remove rows that become empty after standardization
def clean_empty_messages(df):
    cleaned = [custom_standardization(msg).numpy().decode("utf-8").strip() for msg in df["message"].values]
    mask = [msg != "" for msg in cleaned]
    return df[mask]


In [ ]:
# Load train data
train_df = pd.read_csv("train-data.tsv", sep="\t", header=None, names=["label", "message"])
# Load validation data
val_df = pd.read_csv("valid-data.tsv", sep="\t", header=None, names=["label", "message"])

# Drop empty messages
train_df = train_df[train_df['message'].str.strip() != '']
val_df = val_df[val_df['message'].str.strip() != '']

# Encode labels: ham = 0, spam = 1
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
val_df['label'] = val_df['label'].map({'ham': 0, 'spam': 1})
train_df = clean_empty_messages(train_df)
val_df = clean_empty_messages(val_df)

# Extract messages and labels for train and validation sets
train_texts = train_df["message"].values
train_labels = train_df["label"].values
val_texts = val_df["message"].values
val_labels = val_df["label"].values

In [ ]:
train_df['label'].value_counts()

In [ ]:
# --- Oversampling to balance spam and ham ---
ham_df = train_df[train_df['label'] == 0]
spam_df = train_df[train_df['label'] == 1]

spam_oversampled = spam_df.sample(len(ham_df), replace=True, random_state=42)

train_df = pd.concat([ham_df, spam_oversampled])
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

train_texts = train_df["message"].values
train_labels = train_df["label"].values


In [ ]:
train_df['label'].value_counts()

In [ ]:
# Text vectorization layer
# Text Vectorization layer
# Text Vectorization layer
max_features = 10000
sequence_length = 250
vectorizer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)
# Adapt vectorizer on training text only
vectorizer.adapt(train_texts)

In [ ]:
# 1. Prepare dataset
raw_train_ds = tf.data.Dataset.from_tensor_slices((train_texts, train_labels))
raw_train_ds = raw_train_ds.batch(32)

def vectorize_text(text, label):
  return vectorizer(text), label

# 3. Get a batch
text_batch, label_batch = next(iter(raw_train_ds))

# 4. First review and label
first_review = text_batch[0].numpy().decode('utf-8')
first_label = label_batch[0].numpy()

print("Review:", first_review)
print("Label:", "ham" if first_label == 0 else "spam")

# 5. Vectorize first review
vectorized_review, label = vectorize_text(first_review, first_label)
print("Vectorized review:", vectorized_review.numpy())

In [ ]:
print("1287 ---> ",vectorizer.get_vocabulary()[2256])
print(" 313 ---> ",vectorizer.get_vocabulary()[4156])
print('Vocabulary size: {}'.format(len(vectorizer.get_vocabulary())))

In [ ]:
raw_val_ds = tf.data.Dataset.from_tensor_slices((val_texts, val_labels))      # val_texts, val_labels from val_df
# raw_test_ds = tf.data.Dataset.from_tensor_slices((test_texts, test_labels))

In [ ]:
print("1287 ---> ", vectorizer.get_vocabulary()[7632])
print(" 313 ---> ", vectorizer.get_vocabulary()[2256])
print('Vocabulary size: {}'.format(len(vectorizer.get_vocabulary())))


In [ ]:
# Create raw datasets
raw_train_ds = tf.data.Dataset.from_tensor_slices((train_texts, train_labels))
raw_val_ds = tf.data.Dataset.from_tensor_slices((val_texts, val_labels))

train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
# test_ds = raw_test_ds.map(vectorize_text)

In [ ]:
# Add batching, shuffling, and performance optimizations
batch_size = 32
train_ds = train_ds.shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
embedding_dim = 16

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_features + 1, output_dim=embedding_dim, input_length=sequence_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    # Vectorize the input text, expand dims to make batch of 1
    vectorized_text = vectorizer(tf.expand_dims(pred_text, 0))

    # Get model prediction (probability between 0 and 1)
    prediction_prob = model.predict(vectorized_text)[0][0]

    # Convert probability to label
    label = "spam" if prediction_prob >= 0.5 else "ham"

    # Return as list [probability, label]
    return [float(prediction_prob), label]


pred_text = "our new mobile video service is live. just install on your phone to start watching."

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
